## 还是没看完，看了一半啦！**新的一周，*翘了合唱*，继续加油😀**

### Neural Networks

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution kernel
        self.conv1=nn.Conv2d(1,6,3)
        self.conv2=nn.Conv2d(6,16,3)
        # an affine operation: y=Wx+b（矩阵乘法）
        self.fc1=nn.Linear(16*6*6,120) #6*6来自于图像维度
        self.fc2=nn.Linear(120,84)
        self.fc3=nn.Linear(84,10)
    def forward(self,x):
        # Maxpooling over a (2,2) window
        x=F.max_pool2d(F.relu(self.conv1(x)),(2,2))
        # 若window形状是正方形，则可只指定一个数值（边长）
        x=F.max_pool2d(F.relu(self.conv2(x)),2)
        x=x.view(-1,self.num_flat_features(x))
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        x=self.fc3(x)
        return x
    def num_flat_features(self,x):
        size=x.size()[1:]# x[1:]由x[1]到最后一个元素组成
        num_features=1
        for s in size:
            num_features*=s
        return num_features
net=Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


尝试一个随机的 32x32 输入。注意：此网络 （LeNet） 的预期输入大小为 32x32。要在 MNIST 数据集上使用此网络，请将数据集中的图像大小调整为 32x32。

In [10]:
input=torch.randn(1,1,32,32)
out=net(input)
print(out)

tensor([[ 0.1651,  0.0051,  0.0564, -0.0107, -0.1153, -0.0714,  0.0253, -0.0218,
          0.0070,  0.0665]], grad_fn=<AddmmBackward>)


将所有参数的梯度缓存归零并用随机梯度反向传播：

In [11]:
net.zero_grad()
out.backward(torch.randn(1,10))

#### 注意：

`torch.nn`只支持mini-batches。整个`torch.nn`包只支持小批量样本输入，不支持单个样本输入。

例如，`nn.Conv2d`要接受一个4维Tensor*(nSamples*nChannels*Height*Width)*
如果有单个样本，使用`input.unsqueeze(0)`来添加一个伪批量维度。

#### 概括：

`torch.Tensor`-一个多维数组，支持autograd运算如`backward()`，并保存关于该tensor的梯度。

`nn.Module`-神经网络模块。便于封装参数，将其移入GPU，导出，加载等

`nn.Parameter`-一种tensor，当作为属性被赋予一个`Module`时会自动登记为参数*(parameter)*

`autograd.Function`-*autograd运算的forward和backward定义*的实现。每一个`Tensor`运算都会产生至少1个`Function`节点，该节点连接到创建一个`Tensor`的各个函数(*functions*),并编码其历史记录。(Implements *forward and backward definitions of an autograd operation.*Every `Tensor` operation creates at least a single `Function` node that connects to functions that created a `Tensor` and *encodes its history.*)**个人认为即生成计算图，类似*高级人工智能*课件画的那样，或是类似遗传编程画的树**

#### 当前已完成
- [x] 定义神经网络
- [x] 处理输入并调用backward
-[ ]  计算loss
- [ ] 更新网络权重

#### Loss Fuction

目标函数将(output,target)作为输入

nn包中有许多不同的[loss functions](https://pytorch.org/docs/stable/nn.html)。一个简单的loss函数是均方误差`nn.MSELoss`

In [14]:
output = net(input)
target = torch.randn(10) # a dummy target, for example
target = target.view(1,-1) # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(1.1746, grad_fn=<MseLossBackward>)


In [13]:
x=torch.rand(10) # [0,1)之间的均匀分布
y=torch.randn(10)# E=0，D=1的标准正态分布
print(x)
print(y)

tensor([0.4873, 0.2985, 0.9932, 0.8153, 0.6936, 0.1622, 0.2400, 0.9388, 0.3721,
        0.7490])
tensor([-1.4951, -0.9840,  1.0725, -0.2241,  0.7451, -0.2315, -0.0639,  0.4502,
         0.7170, -0.4044])


现在可利用`.grad-fn`属性，查看`loss`的反向传播反向，进而得到计算图。形如：

input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
      -> view -> linear -> relu -> linear -> relu -> linear
      -> MSELoss
      -> loss

当调用`loss.backward()`时，整个计算图(由tensors和Functions构成)计算loss的导数，图中所有的属性为`requires_grad=True`的tensors都会拥有一个`.grad`tensor，该tensor累积梯度信息。

In [19]:
print(loss.grad_fn) # MSELoss
print(loss.grad_fn.next_functions[0][0]) # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0]) #ReLU

#### Backprop

为反向传播error只需调用`loss.backword()`。注意在调用前须将原来的梯度信息置零，否则梯度将会被累加到原来的梯度中。

调用`loss.backward()`,查看conv1的偏置*(bias)*在backward前后的梯度变化。

In [21]:
net.zero_grad() #zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-0.0088, -0.0054, -0.0095, -0.0016, -0.0199,  0.0048])


现在知道怎么使用loss函数了。

**稍后阅读**

   神经网络包包含各种不同的模型和loss函数，他们构成了搭建深度神经网络的组件。全面的列表文档可以查看[这里](https://pytorch.org/docs/stable/nn.html)

#### Update the weights
实践中最简单的参数更新规则是SGD：

   `weight=weight-learning_rate*gradient`

这可以用简单的Python代码实现

In [22]:
learning_rate=0.01
for f in net.parameters():
    f.data.sub_(f.grad.data*learning_rate)

然而，使用神经网络时，往往想使用不同的更新规则比如**SGD, Nesterov-SGD,Adam，RMSProp**等，为此，Pytorch建立了一个小型包`torch.optim`来实现这些方法

In [23]:
import torch.optim as optim

# creat your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad() #zero the gradient buffers
output=net(input)
loss=criterion(output, target)
loss.backward()
optimizer.step() #Does the update

#### 注意

`optimizer.zero_grad()`将梯度缓存置零，只是因为反向传播阶段梯度会被累加。